# **Modelo con preprocesado por Moda y Gaussiano**
Importamos las librerias necesarias para el preprocesado y entrenamiento del modelo



In [ ]:
!pip install kaggle

import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import os
import json

!wget --no-cache -O init.py -q https://raw.githubusercontent.com/rramosp/ai4eng.v1/main/content/init.py
import init; init.init(force_download=False); init.get_weblink()

replicating local resources


# **DESCARGAMOS LOS ARCHIVOS DE LA COMPETENCIA**

In [ ]:
os.environ['KAGGLE_CONFIG_DIR']='/content/'


data = {"username":"jeideralvarez16","key":"cc2c1ad1043aaafcf80d22537833bd56"}
with open('kaggle.json','w') as file:
  json.dump(data, file, indent=4)

!kaggle competitions download -c udea-ai-4-eng-20251-pruebas-saber-pro-colombia\ \(1\).zip

!unzip \*.zip && rm *.zip

!wc *.csv

df_train = pd.read_csv("train.csv")
df_test = pd.read_csv("test.csv")

print ("shape of loaded dataframe", df_train.shape)
print ("shape of loaded dataframe", df_test.shape)

401 Client Error: Unauthorized for url: https://www.kaggle.com/api/v1/competitions/data/download-all/udea-ai-4-eng-20251-pruebas-saber-pro-colombia%20(1).zip
Archive:  udea-ai-4-eng-20251-pruebas-saber-pro-colombia (1).zip
  inflating: submission_example.csv  
  inflating: test.csv                
  inflating: train.csv               
   296787    296787   4716673 submission_example.csv
   296787   4565553  59185250 test.csv
   692501  10666231 143732449 train.csv
  1286075  15528571 207634372 total
shape of loaded dataframe (692500, 21)
shape of loaded dataframe (296786, 20)



#**Funcion clean_data**

1.   Se reemplazan los datos faltantes por la moda
2.   Se normalizan los valores en varias columnas
3.   Se hace una normalización especifica pala la columna ESTU_PRGM_ACADEMICO

In [ ]:
def clean_data(z_copy):
    #Variables Categoricas
  ccols = [i for i in z_copy.columns if not i in z_copy._get_numeric_data()]
  print (ccols)

    #Valor mas repetido
  valor_comun = z_copy[ccols].mode().iloc[0]
  valor_comun

  #Reemplazar datos faltantes con los mas repetidos
  for column in ccols:
    z_copy[column].fillna(valor_comun[column], inplace=True)
  #Modificar la columna valor de la matricula
  z_copy['ESTU_VALORMATRICULAUNIVERSIDAD'].replace(['No pagó matrícula'],[0],inplace=True)
  z_copy['ESTU_VALORMATRICULAUNIVERSIDAD'].replace(['Menos de 500 mil'],[1],inplace=True)
  z_copy['ESTU_VALORMATRICULAUNIVERSIDAD'].replace(['Entre 500 mil y menos de 1 millón'],[2],inplace=True)
  z_copy['ESTU_VALORMATRICULAUNIVERSIDAD'].replace(['Entre 1 millón y menos de 2.5 millones'],[3],inplace=True)
  z_copy['ESTU_VALORMATRICULAUNIVERSIDAD'].replace(['Entre 2.5 millones y menos de 4 millones'],[4],inplace=True)
  z_copy['ESTU_VALORMATRICULAUNIVERSIDAD'].replace(['Entre 4 millones y menos de 5.5 millones'],[5],inplace=True)
  z_copy['ESTU_VALORMATRICULAUNIVERSIDAD'].replace(['Entre 5.5 millones y menos de 7 millones'],[6],inplace=True)
  z_copy['ESTU_VALORMATRICULAUNIVERSIDAD'].replace(['Más de 7 millones'],[7],inplace=True)

  #Modificar la columna horas que trabaja a la semana
  z_copy['ESTU_HORASSEMANATRABAJA'].replace(['0','Menos de 10 horas','Entre 11 y 20 horas','Entre 21 y 30 horas','Más de 30 horas'],[0,1,2,3,4],inplace=True)

  #Modificar la columna estrato de la vivienda
  z_copy['FAMI_ESTRATOVIVIENDA'].replace(['Estrato 1','Estrato 2','Estrato 3','Estrato 4','Estrato 5','Estrato 6','Sin Estrato'],[1,2,3,4,5,6,0],inplace=True)

  #z_copy['RENDIMIENTO_GLOBAL'].replace(['bajo','medio-bajo','medio-alto','alto'],[1,2,3,4],inplace=True)

  #Modificar las columnas de Si y No por 1 y 0
  z_copy["FAMI_TIENEINTERNET"] = z_copy["FAMI_TIENEINTERNET"].replace({'Si': 1, 'No': 0})
  z_copy["ESTU_PAGOMATRICULAPROPIO"] = z_copy["ESTU_PAGOMATRICULAPROPIO"].replace({'Si': 1, 'No': 0})

  #Modificar la columna del Programa academico
  z_copy['ESTU_PRGM_ACADEMICO'] = z_copy['ESTU_PRGM_ACADEMICO'].str.replace('PROFESIONAL EN ', '')
  z_copy['ESTU_PRGM_ACADEMICO'] = z_copy['ESTU_PRGM_ACADEMICO'].str.replace('PROFESIONAL DE ', '')
  z_copy['ESTU_PRGM_ACADEMICO'] = z_copy['ESTU_PRGM_ACADEMICO'].str.replace('PROFESIONAL ', '')
  z_copy['ESTU_PRGM_ACADEMICO'] = z_copy['ESTU_PRGM_ACADEMICO'].str.replace('PROGRAMA ', '')

  z_copy['ESTU_PRGM_ACADEMICO'] = z_copy['ESTU_PRGM_ACADEMICO'].apply(lambda x: 'INGENIERIA' if x.startswith('INGE') else x)
  z_copy['ESTU_PRGM_ACADEMICO'] = z_copy['ESTU_PRGM_ACADEMICO'].apply(lambda x: 'LICENCIATURA' if x.startswith('LIC') or x.startswith('Lic') else x)
  z_copy['ESTU_PRGM_ACADEMICO'] = z_copy['ESTU_PRGM_ACADEMICO'].apply(lambda x: 'ADMINISTRACION' if x.startswith('ADM')   else x)
  z_copy['ESTU_PRGM_ACADEMICO'] = z_copy['ESTU_PRGM_ACADEMICO'].apply(lambda x: 'CONTADURIA' if x.startswith('CONT')   else x)
  z_copy['ESTU_PRGM_ACADEMICO'] = z_copy['ESTU_PRGM_ACADEMICO'].apply(lambda x: 'COMUNICACION' if x.startswith('COMU')   else x)
  z_copy['ESTU_PRGM_ACADEMICO'] = z_copy['ESTU_PRGM_ACADEMICO'].apply(lambda x: 'DEPORTE' if 'DEPORT'  in x else x)
  z_copy['ESTU_PRGM_ACADEMICO'] = z_copy['ESTU_PRGM_ACADEMICO'].apply(lambda x: 'PSICOLOGIA' if 'PSIC' in x else x)
  z_copy['ESTU_PRGM_ACADEMICO'] = z_copy['ESTU_PRGM_ACADEMICO'].apply(lambda x: 'TRABAJO SOCIAL' if 'TRABAJO SOCIAL' in x else x)
  z_copy['ESTU_PRGM_ACADEMICO'] = z_copy['ESTU_PRGM_ACADEMICO'].apply(lambda x: 'DISEÑO' if 'DISE' in x else x)
  z_copy['ESTU_PRGM_ACADEMICO'] = z_copy['ESTU_PRGM_ACADEMICO'].apply(lambda x: 'TERAPIA' if 'TERAP' in x else x)
  z_copy['ESTU_PRGM_ACADEMICO'] = z_copy['ESTU_PRGM_ACADEMICO'].apply(lambda x: 'GASTRONOMIA' if 'GASTRON' in x else x)
  z_copy['ESTU_PRGM_ACADEMICO'] = z_copy['ESTU_PRGM_ACADEMICO'].apply(lambda x: 'VETERINARIA Y ZOOTECNIA' if 'ZOOTECNIA' in x else x)
  z_copy['ESTU_PRGM_ACADEMICO'] = z_copy['ESTU_PRGM_ACADEMICO'].apply(lambda x: 'NUTRICION' if 'NUTRIC' in x else x)
  z_copy['ESTU_PRGM_ACADEMICO'] = z_copy['ESTU_PRGM_ACADEMICO'].apply(lambda x: 'MARKETING' if ( 'MARKETING' in x or 'PUBLICIDAD' in x or 'MERCAD' in x) else x)
  z_copy['ESTU_PRGM_ACADEMICO'] = z_copy['ESTU_PRGM_ACADEMICO'].apply(lambda x: 'MATEMATICAS Y ESTADISTICA' if ('MATE' in x or 'ESTA' in x) else x)
  z_copy['ESTU_PRGM_ACADEMICO'] = z_copy['ESTU_PRGM_ACADEMICO'].apply(lambda x: 'ARTES' if 'ARTE' in x else x)
  z_copy['ESTU_PRGM_ACADEMICO'] = z_copy['ESTU_PRGM_ACADEMICO'].apply(lambda x: 'PERIODISMO' if 'PERIOD' in x else x)
  z_copy['ESTU_PRGM_ACADEMICO'] = z_copy['ESTU_PRGM_ACADEMICO'].apply(lambda x: 'LITERATURA' if 'LITER' in x else x)
  z_copy['ESTU_PRGM_ACADEMICO'] = z_copy['ESTU_PRGM_ACADEMICO'].apply(lambda x: 'MUSICA' if ('MUS' in x or 'MÚS' in x) else x)
  z_copy['ESTU_PRGM_ACADEMICO'] = z_copy['ESTU_PRGM_ACADEMICO'].apply(lambda x: 'ECONOMIA Y NEGOCIOS' if ('ECONO' in x or 'NEG' in x or 'COME' in x or 'FIN' in x) else x)
  z_copy['ESTU_PRGM_ACADEMICO'] = z_copy['ESTU_PRGM_ACADEMICO'].apply(lambda x: 'CIENCIAS DE LA TIERRA' if 'GEO' in x else x)
  z_copy['ESTU_PRGM_ACADEMICO'] = z_copy['ESTU_PRGM_ACADEMICO'].apply(lambda x: 'CIENCIAS AUDIOVISUALES' if ('CINE' in x or 'RADIO' in x or 'TELEV' in x or 'AUDIOV' in x or 'FOTOG' in x) else x)
  z_copy['ESTU_PRGM_ACADEMICO'] = z_copy['ESTU_PRGM_ACADEMICO'].apply(lambda x: 'DERECHO' if 'DERECHO' in x else x)
  z_copy['ESTU_PRGM_ACADEMICO'] = z_copy['ESTU_PRGM_ACADEMICO'].apply(lambda x: 'CIENCIAS POLITICAS' if ( 'POLIT' in x or 'POLÍT' in x) else x)
  z_copy['ESTU_PRGM_ACADEMICO'] = z_copy['ESTU_PRGM_ACADEMICO'].apply(lambda x: 'INGENIERIA' if 'INGE' in x else x)
  z_copy['ESTU_PRGM_ACADEMICO'] = z_copy['ESTU_PRGM_ACADEMICO'].apply(lambda x: 'QUIMICA' if ('QUIM' in x or 'QUÍM' in x or 'QU¿M' in x) else x)
  z_copy['ESTU_PRGM_ACADEMICO'] = z_copy['ESTU_PRGM_ACADEMICO'].apply(lambda x: 'CONSTRUCCION' if 'CONST' in x else x)
  z_copy['ESTU_PRGM_ACADEMICO'] = z_copy['ESTU_PRGM_ACADEMICO'].apply(lambda x: 'TURISMO' if 'TURIS' in x else x)
  z_copy['ESTU_PRGM_ACADEMICO'] = z_copy['ESTU_PRGM_ACADEMICO'].apply(lambda x: 'BIBLIOTECOLOGIA' if 'BIBLIO' in x else x)
  z_copy['ESTU_PRGM_ACADEMICO'] = z_copy['ESTU_PRGM_ACADEMICO'].apply(lambda x: 'FILOSOFIA' if 'FILOS' in x else x)
  z_copy['ESTU_PRGM_ACADEMICO'] = z_copy['ESTU_PRGM_ACADEMICO'].apply(lambda x: 'FILOLOGIA' if 'FILOL' in x else x)
  z_copy['ESTU_PRGM_ACADEMICO'] = z_copy['ESTU_PRGM_ACADEMICO'].apply(lambda x: 'ODONTOLOGIA' if 'ODONTOLOG' in x else x)
  z_copy['ESTU_PRGM_ACADEMICO'] = z_copy['ESTU_PRGM_ACADEMICO'].apply(lambda x: 'BIOLOGIA Y MICROBIOLOGIA' if 'BIOL' in x else x)
  z_copy['ESTU_PRGM_ACADEMICO'] = z_copy['ESTU_PRGM_ACADEMICO'].apply(lambda x: 'ECOLOGIA' if x.startswith('ECOL')   else x)
  z_copy['ESTU_PRGM_ACADEMICO'] = z_copy['ESTU_PRGM_ACADEMICO'].apply(lambda x: 'BACTERIOLOGIA' if 'BACT' in x else x)
  z_copy['ESTU_PRGM_ACADEMICO'] = z_copy['ESTU_PRGM_ACADEMICO'].apply(lambda x: 'CIENCIAS CULTURALES' if 'CULT' in x else x)
  z_copy['ESTU_PRGM_ACADEMICO'] = z_copy['ESTU_PRGM_ACADEMICO'].apply(lambda x: 'INSTRUMENTACION QUIRURGICA' if 'INSTRUMENTACION' in x else x)
  z_copy['ESTU_PRGM_ACADEMICO'] = z_copy['ESTU_PRGM_ACADEMICO'].apply(lambda x: 'CIENCIAS MARITIMAS Y AERONAUTICAS' if ('MILIT' in x or 'NAVALES' in x or 'NAU' in x) else x)


  # Calcula el conteo de cada valor en la columna 'ESTU_PRGM_ACADEMICO'
  conteo_valores = z_copy['ESTU_PRGM_ACADEMICO'].value_counts()

  # Obtén los valores que tienen menos de N registros
  valores_menos_de_20 = conteo_valores[conteo_valores <= 30]
  valores_menos_de_20

  # Reemplaza los valores que tienen menos de N registros con 'OTRO PROGRAMA'
  for valor in valores_menos_de_20.index:
      z_copy.loc[z_copy['ESTU_PRGM_ACADEMICO'] == valor, 'ESTU_PRGM_ACADEMICO'] = 'OTRO PROGRAMA'

  # Identify remaining categorical columns
  remaining_ccols = [i for i in z_copy.columns if not i in z_copy._get_numeric_data()]

  # One-hot encode remaining categorical columns
  z_copy = pd.get_dummies(z_copy, columns=remaining_ccols)

  return z_copy

Aplicamos nuestra función de limpieza a nuestra base de datos

In [ ]:
lentr = len(df_train)

# Definir las columnas de origen, excluyendo la variable objetivo
source_cols = [i for i in df_train.columns if i != "RENDIMIENTO_GLOBAL"]

# Combinar los datos de entrenamiento y prueba para preprocesamiento
all_data = pd.concat((df_train[source_cols], df_test[source_cols]))
all_data.index = range(len(all_data))

# Limpiar los datos usando la función clean_data
all_data = clean_data(all_data)

# Dividir los datos combinados nuevamente en conjuntos de entrenamiento y prueba
Xtrain = all_data.iloc[:lentr].values
ytrain = df_train["RENDIMIENTO_GLOBAL"].values
Xtest = all_data.iloc[lentr:].values

# Imprimir las formas de los conjuntos de entrenamiento y prueba
print("Forma de Xtrain:", Xtrain.shape)
print("Forma de ytrain:", ytrain.shape)
print("Forma de Xtest:", Xtest.shape)

['ESTU_PRGM_ACADEMICO', 'ESTU_PRGM_DEPARTAMENTO', 'ESTU_VALORMATRICULAUNIVERSIDAD', 'ESTU_HORASSEMANATRABAJA', 'FAMI_ESTRATOVIVIENDA', 'FAMI_TIENEINTERNET', 'FAMI_EDUCACIONPADRE', 'FAMI_TIENELAVADORA', 'FAMI_TIENEAUTOMOVIL', 'ESTU_PRIVADO_LIBERTAD', 'ESTU_PAGOMATRICULAPROPIO', 'FAMI_TIENECOMPUTADOR', 'FAMI_TIENEINTERNET.1', 'FAMI_EDUCACIONMADRE']


/tmp/ipython-input-4-1124405769.py:12: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  z_copy[column].fillna(valor_comun[column], inplace=True)
/tmp/ipython-input-4-1124405769.py:21: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  z_copy['ESTU_VALORMATRICULAUNIVERSIDAD'].replace(['Más de 7 millones'],[7

Forma de Xtrain: (692500, 172)
Forma de ytrain: (692500,)
Forma de Xtest: (296786, 172)


# **Entrenamiento del modelo**

In [ ]:
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import cross_val_score
# Inicializar el clasificador Naive Bayes Gaussiano
nb = GaussianNB()

# Realizar validación cruzada en el conjunto de entrenamiento
cv_scores = cross_val_score(nb, Xtrain, ytrain)
print("Puntuaciones de validación cruzada:", cv_scores)

# Entrenar el clasificador en los datos de entrenamiento
nb.fit(Xtrain, ytrain)

# Predecir en el conjunto de prueba
preds_ts = nb.predict(Xtest)
print("Predicciones en el conjunto de prueba:", preds_ts)


Puntuaciones de validación cruzada: [0.31727076 0.31887365 0.31854874 0.31827437 0.31659206]
Predicciones en el conjunto de prueba: ['medio-bajo' 'bajo' 'bajo' ... 'bajo' 'alto' 'bajo']


# **Creación de archivo con las variables necesarias**

In [ ]:
# Crear un DataFrame de envío con el ID y los valores predichos
submission = pd.DataFrame({
    "ID": df_test.ID,
    "RENDIMIENTO_GLOBAL": preds_ts
})

# Mostrar las primeras filas del DataFrame de envío
print(submission.head())

# Guardar el DataFrame de envío en un archivo CSV
print("*************** Crear archivo ************")
submission.to_csv("archivo_gaussiano.csv", index=False)

       ID RENDIMIENTO_GLOBAL
0  550236         medio-bajo
1   98545               bajo
2  499179               bajo
3  782980         medio-bajo
4  785185               bajo
*************** Crear archivo ************
